<center>
    <h1> ILI-286 - Laboratorio #2 </h1>
    <h2> Integración numérica aplicada </h2>
</center>


| Nombre | Rol | Email |
| :----- | :-- | :---- |
| Marco Rojas | 201073005-0 | marco.rojaso@alumnos.usm.cl |
| Hernán Vargas | 201073009-3 | hernan.vargas@alumnos.usm.cl |

## Tabla de contenidos
* [Introducción](#intro)
* [Desarrollo y analisís de resultados:](#desarrollo)
  1. [Integración Impropia](#se1)
* [Concluciones](#Concluciones)
* [Referencias](#Referencias)

<div id='intro' />
## Introducción


<div id='desarrollo'/>
## Desarrollo y analisís de resultados

Debemos comenzar cargando las bibliotecas y datos necesarios. Además se definirán algunas variables que se usarán más adelante.

In [ ]:
import matplotlib
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

<div id='se1'/>
### 1.- Integración Impropia



<div id='de1'/>
#### 1.1) Cambio de variable propuesto
Si se quiere llevar la variable t desde el intervalo de integración $[-\infty,x]$ a el $[a,b]$ se hace un cambio de 
variable utilizando la nueva variable u definida por
$$ u = -\displaystyle \frac{b-a}{t-x-1} + a $$ 
$$ \Rightarrow t = \displaystyle \frac{b + a \, x + u (1+x)} {a-u} $$
$$ dt = \displaystyle \frac{b-a}{(u-a)^2} $$ 

In [ ]:
plt.figure(figsize=(12,6))

a = -1
b= 1
x = 30
t = np.linspace(-2*x, x, 200)
plt.plot(t, -(b-a)/(t-x-1)+a)
plt.title('Cambio de variable asumiendo a='+str(a) + " b=" + str(b) + " x=" + str(x))
plt.xlabel("variable t")
plt.ylabel("variable u")
plt.show()

$$ f(x) = \int_{-\infty}^x \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{t^2}{2}} \, dt = 
\int_{a}^b \displaystyle \frac {1}{\sqrt{2 \pi}} \, e^{-  \frac{{(-b-a\,x + u(1+x)})^2}{2(u-a)^2}} \, \frac{b-a}{(u-a)^2} du $$

<div id='de2'/>
#### 1.2) Implementaciones de algoritmos de integración


In [ ]:
def midpoint(fun, N, a, b):
    #vectorizamos la funcion para poder llamarla con muchos puntos a la vez
    f_vec = np.vectorize(fun)
    #obtenemos N intervalos y N+1 puntos equidistantes entre a y b
    x = np.linspace(a, b, N+1)
    #obtenemos el diferencial
    dx = x[1]-x[0]
    # a todos los puntos menos el final le sumamos 0.5 por el diferencial
    midpoints = x[:-1] + 0.5*dx
    #obtenemos las evaluaciones de los puntos
    midpoint_values = f_vec(midpoints)
    #obtenemos el resultado final
    int_val = sum(midpoint_values*dx)
    return int_val

def trapezoid(fun, N, a, b):
    f_vec = np.vectorize(fun) 
    x = np.linspace(a, b, N+1)
    dx = x[1]-x[0]
    xmiddle = x[1:-1]
    int_val = 0.5*dx*(f_vec(x[0])+2*sum(f_vec(xmiddle))+f_vec(x[-1]))
    return int_val

def simpsons(fun, N, a, b):
    f_vec = np.vectorize(fun)
    x = np.linspace(a, b, N+1)
    if N%2==1:
        print("Simpsons solo aplica con catntidad de intervalos pares.")
        return
    dx = x[1]-x[0]
    #obtenemos elementos de la izquierda partiendo desde el primero de dos en dos hasta el antepenultimo
    xleft   = x[:-2:2]
    #obtenemos elementos centrales partiendo desde el segundo de dos en dos
    xmiddle = x[1::2]
    #obtenemos elementos de la derecha partiendo desde el tercero de dos en dos
    xright  = x[2::2]
    #los sumamos y multiplicamos, xleft contiene casi todos los elementos presentes 
    #en xright menos el último por lo que no lo multiplicamos por 2
    int_val = sum((f_vec(xleft)+4*f_vec(xmiddle)+f_vec(xright))*dx/3)
    return int_val

def gaussianquad(fun, N, a, b):
    f_vec = np.vectorize(fun)
    x, w = gaussian_nodes_and_weights(N, a, b)
    int_val = sum( w * f_vec(x) )
    return int_val

def gaussian_nodes_and_weights(N, a, b):
    if N==1: 
        return np.array([1]), np.array([2])
    beta = .5 / np.sqrt(1.-(2.*np.arange(1.,N))**(-2))
    T = np.diag(beta,1) + np.diag(beta,-1)
    D, V = np.linalg.eigh(T)
    x = D
    #Reescalamiento
    x = .5 * ( (b-a)*x + b + a)
    w = 2*V[0,:]**2
    w = .5*(b-a)*w
    return x, w

N = 10
a = 1
b= 4
fun = lambda x : x**2 + x**1+5
print(midpoint(fun, N, a, b))
print(trapezoid(fun, N, a, b))
print(simpsons(fun, N, a, b))
print(gaussianquad(fun, N, a, b))

<div id='de3'/>
#### 1.3) Análisis de los métodos de integración

In [ ]:
N = 50
a = -1
b= 1
x = 10**2
fun = lambda t: (np.e**((-t**2)/2))/ np.sqrt(2*np.pi)
fun2 = lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)
print(midpoint(fun, N, a, b))
print(trapezoid(fun, N, a, b))
print(simpsons(fun, N, a, b))
print(gaussianquad(fun, N, a, b))
print(gaussianquad(fun2, N, a, b))
print(gaussianquad(fun2, N, -1, 1))

#matriz de 50 (alphas), 4 (métodos) y 5 h distintos
res = np.zeros([50, 4, 5])
delta = 10**-10
#for entre 0 y 49
for i in range(0,50):
    m =0;
    for n in [10, 50, 100, 500, 1000]:
        #a = 2
        #b = 3
        #fun2 = lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)
        res[i, 0, m] = midpoint(fun2, n, a, b)
        res[i, 1, m] = trapezoid(fun2, n, a+delta, b-delta)
        res[i, 2, m] = simpsons(fun2, n, a+delta, b-delta)
        #a = -1
        #b = 1
        #fun2 = lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)
        res[i, 3, m] = gaussianquad(fun2, n, a, b)
        m = m+1;
    
print(res)


<div id='de4'/>
#### 1.4) Caso $x = 0$

In [ ]:
N = 500
a = -2
b= -9
x = 0
fun = lambda t: (np.e**((-t**2)/2))/ np.sqrt(2*np.pi)
fun2 = lambda u: (np.e**(-(( (-b-a*x + u*(1+x)) / (u-a)  )**2 )/2)  * ((b-a)/(u-a)**2)  )/ np.sqrt(2*np.pi)
print(midpoint(fun2, N, a, b))
#print(trapezoid(fun2, N, a, b))
#print(simpsons(fun2, N, a, b))
print(gaussianquad(fun2, N, a, b))



<div id='de5'/>
#### 1.5) Caso $ x = +\infty$

Si se quiere llevar la variable t desde el intervalo de integración $[-\infty,\infty]$ a el $[a,b]$ se hace un cambio de 
variable utilizando la nueva variable u definida por
$$ u = \displaystyle \frac{b-a}{\pi} \, \tan^{-1} (t) + \displaystyle \frac{b-a}{2} +a $$ 
$$ \Rightarrow  t = \displaystyle \cot ( \frac {\pi (u-b)}{a-b}) $$
$$ dt = - \displaystyle \frac {\csc^2 ( \displaystyle \frac {\pi (u-b)}{a-b} )} {a-b} \, du $$

In [ ]:
plt.figure(figsize=(12,6))

a = -1
b= 1
x = np.inf;
t = np.linspace(-10, 10, 200)
plt.plot(t, ((b-a)*np.arctan(t))/np.pi + (b-a)/2 + a)
plt.title('Cambio de variable asumiendo a='+str(a) + " b=" + str(b) + " x=" + str(x))
plt.xlabel("variable t")
plt.ylabel("variable u")
plt.show()


N = 500
a = -2
b= -9
delta = 10**-10
    
fun = lambda t: (np.e**((-t**2)/2))/ np.sqrt(2*np.pi)
fun2 = lambda u: np.e**(-((  np.tan(np.pi * (u-b)/(a-b)  )**-2 )/2))  * (  ((np.sin(np.pi * (u-b)/(a-b)))**-2)/(a-b)   )/ np.sqrt(2*np.pi)
print(midpoint(fun2, N, a, b))
print(trapezoid(fun2, N, a+delta, b-delta))
print(simpsons(fun2, N, a+delta, b-delta))
print(gaussianquad(fun2, N, a, b))


## Conclusiones



## Referencias
<div id='ref1'\> [1] [NumPy v1.10 Manual](http://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.solve.html) numpy.linalg.solve. *The solutions are computed using LAPACK routine _gesv*. Revisado 30/10/2015

<div id='ref2'\> [2] [LAPACK Benchmark](http://www.netlib.org/lapack/lug/node71.html). Ver tabla 3.13. Revisado el 30/10/2015